# 🔬 胚胎著床預測 - Focal Loss 實驗

> EXP-003: 使用 Focal Loss 處理類別不平衡

**目標**：測試 Focal Loss 是否能同時保持高 AUC 和提升 Recall

**改動**：將 CrossEntropyLoss + 類別加權 改為 Focal Loss

---

## Focal Loss 是什麼？

**白話解釋**：
- 原本的 Loss：不管 AI 猜得多有信心，錯了就罰一樣多
- Focal Loss：AI 已經很有信心且猜對的，少罰一點；AI 不確定的，多罰一點

**效果**：讓 AI 專注在「難分辨」的樣本上，而不是一直學簡單的

---

## 1️⃣ 環境設置

In [ ]:
# 掛載 Google Drive
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 已掛載")
else:
    print("ℹ️ 不在 Colab 環境，跳過掛載")

In [ ]:
# 匯入套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, 
    recall_score, f1_score, confusion_matrix
)

# 設定隨機種子
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# 檢查 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ 使用裝置: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ 設定資料路徑

In [ ]:
if IN_COLAB:
    DATA_ROOT = '/content/drive/MyDrive/hvwc23'
else:
    DATA_ROOT = '../data/raw/hvwc23'

TRAIN_DIR = os.path.join(DATA_ROOT, 'train')
TRAIN_CSV = os.path.join(DATA_ROOT, 'train.csv')

print("📁 檢查資料路徑...")
print(f"   訓練資料夾: {os.path.exists(TRAIN_DIR)}")
print(f"   訓練 CSV:   {os.path.exists(TRAIN_CSV)}")

## 3️⃣ 載入與準備資料

In [ ]:
# 載入資料
train_df = pd.read_csv(TRAIN_CSV)
print(f"📊 訓練資料筆數: {len(train_df)}")

# 分割
train_data, val_data = train_test_split(
    train_df, test_size=0.2, random_state=SEED, stratify=train_df['Class']
)
print(f"   訓練集: {len(train_data)} 張")
print(f"   驗證集: {len(val_data)} 張")

In [ ]:
# 資料轉換（與之前相同）
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ 資料轉換定義完成")

In [ ]:
# Dataset
class EmbryoDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.img_dir, row['Image'])
        image = Image.open(img_path).convert('RGB')
        label = row['Class']
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = EmbryoDataset(train_data, TRAIN_DIR, train_transform)
val_dataset = EmbryoDataset(val_data, TRAIN_DIR, val_transform)

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print("✅ DataLoader 建立完成")

## 4️⃣ 定義 Focal Loss

🔄 **這是與 EXP-001、EXP-002 的主要差異**

In [ ]:
# =============================================
# 🔄 Focal Loss 定義
# =============================================

class FocalLoss(nn.Module):
    """
    Focal Loss - 讓模型專注在難分類的樣本
    
    參數：
    - alpha: 類別權重（處理不平衡）
    - gamma: 聚焦程度（越大越專注在難的樣本）
    
    公式：FL = -alpha * (1-p)^gamma * log(p)
    - 當 p 接近 1（很有信心且猜對）：(1-p)^gamma 很小，loss 變小
    - 當 p 接近 0（很沒信心或猜錯）：(1-p)^gamma 接近 1，loss 維持
    """
    def __init__(self, alpha=None, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # 類別權重
        self.gamma = gamma  # 聚焦參數
    
    def forward(self, inputs, targets):
        # 計算基本的 CrossEntropy
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
        # 計算預測機率
        pt = torch.exp(-ce_loss)
        
        # 計算 Focal 權重
        focal_weight = (1 - pt) ** self.gamma
        
        # 加入類別權重
        if self.alpha is not None:
            alpha_weight = self.alpha[targets]
            focal_loss = alpha_weight * focal_weight * ce_loss
        else:
            focal_loss = focal_weight * ce_loss
        
        return focal_loss.mean()

print("✅ Focal Loss 定義完成")
print()
print("📊 Focal Loss vs CrossEntropyLoss 差異：")
print("   CrossEntropyLoss: 所有樣本一視同仁")
print("   Focal Loss: 專注在難分類的樣本（gamma 越大越專注）")

## 5️⃣ 建立模型

In [ ]:
# 使用 EfficientNet-B0（因為 AUC 較高）
model = models.efficientnet_b0(weights='IMAGENET1K_V1')
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)
model = model.to(device)

print("✅ 模型建立完成: EfficientNet-B0")

In [ ]:
# 計算類別權重（用於 Focal Loss）
class_counts = train_data['Class'].value_counts().sort_index()
total = len(train_data)
alpha = torch.tensor([total / (2 * class_counts[i]) for i in range(2)], dtype=torch.float32)
alpha = alpha.to(device)

print(f"📊 類別權重 (alpha):")
print(f"   Class 0: {alpha[0]:.4f}")
print(f"   Class 1: {alpha[1]:.4f}")

In [ ]:
# =============================================
# 🔄 使用 Focal Loss（與 EXP-001、EXP-002 不同）
# =============================================

# Focal Loss 參數
GAMMA = 2.0  # 聚焦程度，通常用 2

criterion = FocalLoss(alpha=alpha, gamma=GAMMA)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

print("✅ 訓練設定完成")
print(f"   損失函數: Focal Loss (gamma={GAMMA})")
print(f"   優化器: Adam (lr=0.001)")
print()
print("📊 與之前實驗的差異：")
print("   EXP-001: CrossEntropyLoss + 類別加權")
print("   EXP-002: CrossEntropyLoss + 類別加權")
print("   EXP-003: Focal Loss + 類別加權 ← 新的！")

## 6️⃣ 訓練模型

In [ ]:
# 訓練函數
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), correct / total

# 驗證函數
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except:
        auc = 0.0
    
    return running_loss / len(loader), accuracy, auc, all_preds, all_probs, all_labels

print("✅ 訓練函數定義完成")

In [ ]:
# 開始訓練
NUM_EPOCHS = 30

history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [], 'val_auc': []
}

best_auc = 0.0
best_epoch = 0

print("="*60)
print("🚀 開始訓練 (Focal Loss)")
print("="*60)

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_auc, _, _, _ = validate(model, val_loader, criterion, device)
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_auc'].append(val_auc)
    
    if val_auc > best_auc:
        best_auc = val_auc
        best_epoch = epoch + 1
        torch.save(model.state_dict(), 'best_model_focal.pth')
    
    print(f"Epoch {epoch+1:02d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}")

print("="*60)
print(f"✅ 訓練完成！最佳 AUC: {best_auc:.4f} (Epoch {best_epoch})")
print("="*60)

## 7️⃣ 視覺化訓練過程

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['train_loss'], label='Train', marker='o', markersize=3)
axes[0].plot(history['val_loss'], label='Validation', marker='o', markersize=3)
axes[0].set_title('Loss (Focal Loss)')
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['train_acc'], label='Train', marker='o', markersize=3)
axes[1].plot(history['val_acc'], label='Validation', marker='o', markersize=3)
axes[1].set_title('Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(history['val_auc'], label='EXP-003 (Focal)', marker='o', markersize=3, color='green')
axes[2].axhline(y=best_auc, color='green', linestyle='--', alpha=0.5)
axes[2].axhline(y=0.7986, color='blue', linestyle=':', label='EXP-001: 0.7986')
axes[2].axhline(y=0.8283, color='orange', linestyle=':', label='EXP-002: 0.8283')
axes[2].set_title('Validation AUC')
axes[2].set_xlabel('Epoch')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves_focal.png', dpi=150)
plt.show()

## 8️⃣ 最終評估

In [ ]:
# 載入最佳模型
model.load_state_dict(torch.load('best_model_focal.pth'))
print(f"✅ 載入最佳模型 (Epoch {best_epoch})")

# 最終驗證
val_loss, val_acc, val_auc, all_preds, all_probs, all_labels = validate(
    model, val_loader, criterion, device
)

In [ ]:
# 計算詳細指標
precision_1 = precision_score(all_labels, all_preds, pos_label=1)
recall_1 = recall_score(all_labels, all_preds, pos_label=1)
f1_1 = f1_score(all_labels, all_preds, pos_label=1)

print("="*60)
print("📊 EXP-003 最終評估結果")
print("="*60)
print()
print(f"✅ Accuracy:  {val_acc:.4f} ({val_acc*100:.1f}%)")
print(f"✅ AUC-ROC:   {val_auc:.4f}")
print(f"✅ Precision: {precision_1:.4f}")
print(f"✅ Recall:    {recall_1:.4f}")
print(f"✅ F1-Score:  {f1_1:.4f}")
print()
print("📊 三次實驗比較：")
print(f"{'實驗':<10} {'模型':<15} {'損失函數':<20} {'AUC':<8} {'Recall':<8} {'Precision':<8}")
print("-" * 75)
print(f"{'EXP-001':<10} {'ResNet18':<15} {'CE + 加權':<20} {'0.7986':<8} {'0.7600':<8} {'0.5000':<8}")
print(f"{'EXP-002':<10} {'EfficientNet':<15} {'CE + 加權':<20} {'0.8283':<8} {'0.6400':<8} {'0.5714':<8}")
print(f"{'EXP-003':<10} {'EfficientNet':<15} {'Focal Loss':<20} {val_auc:<8.4f} {recall_1:<8.4f} {precision_1:<8.4f}")

In [ ]:
# 混淆矩陣
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['預測: 不著床', '預測: 著床'],
            yticklabels=['實際: 不著床', '實際: 著床'])
plt.title('混淆矩陣 - Focal Loss (EXP-003)')
plt.ylabel('實際值')
plt.xlabel('預測值')

tn, fp, fn, tp = cm.ravel()
plt.figtext(0.5, -0.05, 
            f'TN={tn} | FP={fp} | FN={fn} | TP={tp}', 
            ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('confusion_matrix_focal.png', dpi=150, bbox_inches='tight')
plt.show()

## 9️⃣ 結果摘要

In [ ]:
print("="*60)
print("📋 EXP-003 結果摘要（請複製此區塊回報）")
print("="*60)
print(f"""
## 實驗設定
- 模型: EfficientNet-B0 (預訓練)
- 損失函數: Focal Loss (gamma={GAMMA})
- 訓練集: {len(train_data)} 張
- 驗證集: {len(val_data)} 張
- Batch size: {BATCH_SIZE}
- Epochs: {NUM_EPOCHS}

## 結果
| 指標 | 數值 |
|------|------|
| Accuracy | {val_acc:.4f} ({val_acc*100:.1f}%) |
| AUC-ROC | {val_auc:.4f} |
| Precision (著床) | {precision_1:.4f} |
| Recall (著床) | {recall_1:.4f} |
| F1-Score (著床) | {f1_1:.4f} |

## 混淆矩陣
- TN (猜對不著床): {tn}
- FP (誤報): {fp}
- FN (漏掉): {fn}
- TP (猜對著床): {tp}

## 三次實驗比較
| 實驗 | 模型 | 損失函數 | AUC | Recall | Precision |
|------|------|----------|-----|--------|----------|
| EXP-001 | ResNet18 | CE+加權 | 0.7986 | 0.7600 | 0.5000 |
| EXP-002 | EfficientNet | CE+加權 | 0.8283 | 0.6400 | 0.5714 |
| EXP-003 | EfficientNet | Focal Loss | {val_auc:.4f} | {recall_1:.4f} | {precision_1:.4f} |
""")